In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip "/content/drive/MyDrive/ibm/Digital Naturalist Dataset.zip"

Archive:  /content/drive/MyDrive/ibm/Digital Naturalist Dataset.zip
   creating: Digital Naturalist Dataset/
   creating: Digital Naturalist Dataset/Bird/
   creating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (1).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (10).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (11).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (2).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (3).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (4).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (5).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (6).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great 

In [3]:
cd /content/Digital Naturalist Dataset

/content/Digital Naturalist Dataset


In [4]:
ls

Bird/  Flower/  Mammal/


 **DataAugmantation**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from os import listdir
import time

In [6]:
def hms_string (sec_elapsed):
  h = int(sec_elapsed / (60 * 60))
  m = int((sec_elapsed % (60*60)) / 60)
  S = sec_elapsed % 60
  return f"{h}: {m}: {round (S,1)}"
def augment_data( file_dir, n_generated_samples, save_to_dir):
  data_gen = ImageDataGenerator (rotation_range=30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.15,
                                 shear_range=0.25,
                                 zoom_range = 0.2,
                                 horizontal_flip=True,
                                 vertical_flip=False,
                                 fill_mode='nearest',
                                 brightness_range=(0.5,1.2))
  for filename in listdir(file_dir):
        image = cv2.imread(file_dir + '/' + filename)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = image.reshape((1,)+image.shape)
        save_prefix = 'aug_' + filename[:-4]    
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir,
                                   save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i > n_generated_samples:
                break

In [7]:
             
start_time= time.time()

augmented_data_path ='/content/drive/MyDrive/ibm/Augmentation data'

augment_data(file_dir='/content/Digital Naturalist Dataset/Bird/Great Indian Bustard Bird', n_generated_samples=42, save_to_dir=augmented_data_path+'/Bird/GIB_AUG')
augment_data(file_dir='/content/Digital Naturalist Dataset/Bird/Spoon Billed Sandpiper Bird', n_generated_samples=42, save_to_dir=augmented_data_path+'/Bird/SPS_AGU')

augment_data(file_dir='/content/Digital Naturalist Dataset/Flower/Corpse Flower', n_generated_samples=49, save_to_dir=augmented_data_path+'/Flower/Cropse_AUG')
augment_data(file_dir='/content/Digital Naturalist Dataset/Flower/Lady Slipper Orchid Flower', n_generated_samples=49, save_to_dir=augmented_data_path+'/Flower/LS_Orchid_AUG')

augment_data(file_dir='/content/Digital Naturalist Dataset/Mammal/Pangolin Mammal', n_generated_samples=47, save_to_dir=augmented_data_path+'/Mammal/Pangolin_AUG')
augment_data(file_dir='/content/Digital Naturalist Dataset/Mammal/Senenca White Deer Mammal', n_generated_samples=47, save_to_dir=augmented_data_path+'/Mammal/SW_Deer_AUG')

end_time = time.time()
execution_time =(end_time - start_time)
print(f"elapsed time: {hms_string(execution_time)}")

elapsed time: 0: 5: 27.8


**Libraries**

In [8]:
import numpy as np

#Importing libraries required for the model
import tensorflow as tf
import keras
import keras.backend as K

from keras.optimizers import SGD, Adam, Adagrad, RMSprop 
from keras.applications import*
from keras.preprocessing import*
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization, Dropout 
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator as idm
from sklearn.model_selection import train_test_split

#For plotting charts used for data visualizations 
import matplotlib.pyplot as plt

#Libraries for Locating and loading data
import glob
from PIL import Image
import os
from os import listdir

import warnings

**Make a list of path to all folders where you have the data**

In [9]:
#Setting path to our dataset folder 
dirName = '/content/drive/MyDrive/ibm/Augmentation data'
folders = listdir(dirName)
#Getting the names for all the folders containing data 
def getListOfDrive (dirName): 
  # create a list of sub directories and files (if any)
# names in the given directory
  ListOfDrive = os.listdir (dirName)
  allDrive = list()
  for fol_name in ListOfDrive: 
    fullPath = os.path.join(dirName, fol_name)
    allDrive.append(fullPath)
  
  return allDrive

Folders = getListOfDrive(dirName)
len(Folders)
subfolders = []
for num in range(len (Folders)):
    sub_fols = getListOfDrive(Folders [num]) 
    subfolders+=sub_fols 
#Now, the subfolders contains the address to all our data folders for each class
subfolders

['/content/drive/MyDrive/ibm/Augmentation data/Bird/SPS_AGU',
 '/content/drive/MyDrive/ibm/Augmentation data/Bird/GIB_AUG',
 '/content/drive/MyDrive/ibm/Augmentation data/Flower/LS_Orchid_AUG',
 '/content/drive/MyDrive/ibm/Augmentation data/Flower/Cropse_AUG',
 '/content/drive/MyDrive/ibm/Augmentation data/Mammal/SW_Deer_AUG',
 '/content/drive/MyDrive/ibm/Augmentation data/Mammal/Pangolin_AUG']

**Loading images into machine understandable data**

In [10]:
X_data = []
Y_data = []

id_no = 0
found = []
for paths in subfolders:
  drive = glob.glob ('/content/drive/MyDrive/ibm/Augmentation data/*.jpg')
  found.append((paths.split('//')[-1], paths.split('//')[-1]))

  for MyDrive in drive:
    img = Image.open (MyDrive)
    img =img.resize(( 150, 150), Image.ANTIALIAS)
    img = np.array(img)
    if img.shape == ( 150, 150, 3):
      X_data.append (img)
      Y_data.append (id_no)
id_no+= 1

**Data Spliting into Train and Test**

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range = 0.2, zoom_range=0.2 ,horizontal_flip= True, vertical_flip=True)


In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
X_train = train_datagen.flow_from_directory(r"/content/Digital Naturalist Dataset",target_size=(64,64), batch_size=32, class_mode="categorical")

Found 138 images belonging to 3 classes.


In [14]:
X_test = test_datagen.flow_from_directory(r"/content/Digital Naturalist Dataset",target_size=(64,64), batch_size=32, class_mode="categorical")

Found 138 images belonging to 3 classes.


In [15]:
X_train.class_indices

{'Bird': 0, 'Flower': 1, 'Mammal': 2}